In [ ]:
# 使用できるGPUの確認
!nvidia-smi

In [ ]:
# KFTTのダウンロード
!wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz

In [ ]:
# 解凍
!tar -zxvf kftt-data-1.0.tar.gz

In [ ]:
# fairseqのインストール
# 再起動を求められることがあるのでその時は再起動してください。
!pip install fairseq

In [1]:
def limiter(data_dir, opt_path, src_data, tgt_data, n=50):
  '''limiter
  コーパスの最大文長を制限する。
  ----------------------------------------
  引数
  data_dir : コーパスのディレクトリ
  src_data : 原言語のデータ
  tgt_data : 目的言語のデータ
  n : 最大文長
  ----------------------------------------
  '''
  # 書き込み用のデータファイルを作成する。
  with open(f'{data_dir}/{opt_path}/{src_data}', 'w', encoding='utf8') as wsrc:
    with open(f'{data_dir}/{opt_path}/{tgt_data}', 'w', encoding='utf8') as wtgt:
      # 元のデータファイルを読み込む
      with open(f'{data_dir}/kyoto-{src_data}', encoding='utf8') as rsrc:
        with open(f'{data_dir}/kyoto-{tgt_data}', encoding='utf8') as rtgt:
          # それぞれ一行ずつ読み込みリストに格納する。
          src_lines = rsrc.read().strip().split('\n')
          tgt_lines = rtgt.read().strip().split('\n')
          for src, tgt in zip(src_lines, tgt_lines):
            # 最大文長がn以下のもののみ書き込む。
            if len(src.split()) <= n and len(tgt.split()) <= n:
              wsrc.write(src + '\n')
              wtgt.write(tgt + '\n')

In [2]:
# 文長制限付きのファイルを格納するフォルダを作成。
!mkdir kftt-data-1.0/data/tok/kyoto-limited.en-ja

In [3]:
## 文長制限するファイルの一覧
files = ['train', 'dev', 'test']
for f in files:
  # 最大文長10とする(変更可能)
  limiter('kftt-data-1.0/data/tok', f'kyoto-limited.en-ja', f'{f}.en', f'{f}.ja', n=10)

In [4]:
# preprocessで参照するフォルダ
TEXT = "kftt-data-1.0/data/tok/kyoto-limited.en-ja"

In [5]:
# ファイル名がvalidでないとErrorになるので変更する。
!mv $TEXT/dev.en $TEXT/valid.en
!mv $TEXT/dev.ja $TEXT/valid.ja

In [6]:
# preprocessの保存先のファイルを作成。
!mkdir kftt-data-1.0/data/data-bin

In [ ]:
# preprocessを実行
!fairseq-preprocess --source-lang en --target-lang ja \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir kftt-data-1.0/data/data-bin/kyoto-limited.en-ja

In [ ]:
# trainを実行。
!fairseq-train kftt-data-1.0/data/data-bin/kyoto-limited.en-ja \
    --arch transformer \
    --optimizer adam \
    --adam-betas '(0.9, 0.98)' \
    --clip-norm 1.0 \
    --lr-scheduler inverse_sqrt \
    --warmup-init-lr 1e-07 \
    --warmup-updates 1000 \
    --lr 0.01 \
    --min-lr 1e-09 \
    --dropout 0.1 \
    --weight-decay 0.0 \
    --no-epoch-checkpoints \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --max-tokens 2500 \
    --max-update 28000 \
    --save-dir checkpoints/ \
    --max-epoch 20 \
    --log-format simple \
    --log-interval 5 \
    --ddp-backend no_c10d \
    --update-freq 32 \
    --seed 42

In [18]:
# モデルの保存先のファイルを参照
!ls checkpoints/

checkpoint_best.pt  checkpoint_last.pt


In [ ]:
# 翻訳を実行(モデルの評価までしてくれます)
!fairseq-generate kftt-data-1.0/data/data-bin/kyoto-limited.en-ja \
    --path checkpoints/checkpoint_last.pt \
    --batch-size 128 --beam 5